In [1]:
# 모듈 임포트
import os, json
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob

In [2]:
# 파일 경로 리스트
train_json_list = glob("C:\\data\\train\\*\\*.json")
train_img_list = glob("C:\\data\\train\\*\\*.png")
val_json_list = glob("C:\\data\\valid\\*\\*.json")
val_img_list = glob("C:\\data\\valid\\*\\*.png")

In [3]:
# json 파일에서 bbox 정보 리스트로 불러오기
def bbox_list(json_list):
    tmp = []
    for item in json_list:
        with open(item, 'r', encoding='utf8') as f:
            json_data = json.load(f)
            json_data = json_data['annotations']
            tmp.append(json_data)
    return tmp

# bbox 정보 리스트를 DataFrame으로 작성
def make_df(bbox_list):
    tmp_df = pd.DataFrame()
    for i in range(len(bbox_list)):
        tmp = pd.DataFrame(bbox_list[i])
        tmp['fileno'] = i
        tmp_df = pd.concat([tmp_df, tmp])
        tmp_df = tmp_df[['fileno', 'label', 'points']]
    return tmp_df

# bbox 정보 리스트와 작성한 DataFrame을 바탕으로 yolo 좌표를 추출하고
# DataFrame에 결합
def yolo_trans_df(bbox_list,df):
    tmp_list = []
    for i in range(len(bbox_list)):
        tmp = df[df.fileno == i]
        for j in range(len(tmp)):
            top_left = tmp.points[j][0]
            top_right = tmp.points[j][1]
            bottom_right = tmp.points[j][2]
            bottom_left = tmp.points[j][3]

            w = bottom_right[0] - bottom_left[0]
            h = bottom_right[1] - top_right[1]
            x = bottom_right[0] - w/2
            y = bottom_right[1] - h/2

            a1 = x / 1920
            a2 = y / 1080
            a3 = w / 1920
            a4 = h / 1080

            tmp_list.append(str(a1) + ' ' + str(a2) + ' ' + str(a3) + ' ' + str(a4))

    yolo = pd.DataFrame(tmp_list)
    df = df.reset_index(drop=True)
    df = pd.concat([df, yolo], axis=1)
    
    return df

# 중복값을 제거하여 카테고리를 딕셔너리로 생성
def make_category(json_list, bbox_list):
    cat = []
    for i in range(len(bbox_list)):
        for j in range(len(bbox_list[i])):
            cat.append(bbox_list[i][j]['label'])
        cat_list = sorted(list(set(cat)))
        cat_dic = {string : idx for idx,string in enumerate(cat_list)}

    return cat_dic

# yolo 라벨을 txt로 저장
def yolo_label(img_list, df, cat_dic, directory):
    img_nm = []
    for i in range(len(img_list)):
        img_nm.append(img_list[i].split('\\')[-1].split('.')[0].split('_')[-1])

    for i in range(len(img_list)):
        f = open(directory + f"{img_nm[i]}.txt", "w")
        for j in range(len(df[df.fileno == i])):
            tmp = df[df.fileno == i]
            tmp = tmp.reset_index(drop=True)
            f.write(f"{cat_dic[tmp.loc[j].label]} {tmp[0][j]}\n")
    f.close()

In [4]:
tr_bbox = bbox_list(train_json_list)
val_bbox = bbox_list(val_json_list)

tr_df = make_df(tr_bbox)
val_df = make_df(val_bbox)

tr_df = yolo_trans_df(tr_bbox, tr_df)
val_df = yolo_trans_df(val_bbox, val_df)

cat_dic = make_category(train_json_list, tr_bbox)


train_directory = "C:\\data\\train\\labels\\"
valid_directory = "C:\\data\\valid\\labels\\"

yolo_label(train_img_list, tr_df, cat_dic, train_directory)
yolo_label(val_img_list, val_df, cat_dic, valid_directory)

In [5]:
cat_dic

{'목적차량(특장차)': 0, '보행자': 1, '이륜차': 2, '일반차량': 3}